# Huawei Research France

## Transfer learning on home network: 

## Build a transfer learning solution for home network failure prediction

_Aladin Virmaux, Illyyne Saffar, Jianfeng Zhang, Balázs Kégl (Huawei Research, Noah's Ark Laboratory, France)_

## Introduction

Optical access network (OAN) is a mainstream Home Broadband Access Network solution around the world. It connects terminal subscribers to their service provider. Network failures affect both the quality of the service (QoS) and the user experience (the quality of experience QoE). To reduce the damage, it is important to predict in advance the network failures and fix them in time. Machine learning (ML) algoritms have been widely used as a solution to build these failure prediction models. However, most ML models are data-specific and are prone to degradation when the data distribution changes. This year's first Huawei France data challenge aims at solving this problem. You will receive a labeled optical access network dataset from a city we call "A" (which we name the _source_ domain) and a mostly unlabeled  dataset from a city "B" (which we name the _target_ domain). You are asked to build a transfer learning solution using the labaled source data plus the unlabeled target data to train a failure prediction model for city B. It is an **unsupervised domain adaptation (UDA)** problem. To be precise, we do include a small number of labeled target points in the training set so we can call the setup "few-shot UDA" or "semi-supervised domain adaptation". 

Additional challenges will come from

1. **missing values**: there are a lot of missing values in the data;
2. **time series sensor data**;
3. **class imbalance**: network failures are rare, thus it is very imbalanced classification problem. 


## Context

Transmission technologies have evolved to integrate optical technologies even in access networks, as close as possible to the subscriber. Currently, fiber optic is the transmission medium par excellence due to its ability to propagate signal over long distances without regeneration, its low latency, and its very high bandwidth. Optical fiber, initially deployed in very long distance and very high speed networks, is now tending to be generalized to offer more consumer services in terms of bandwidth. These are FTTH technologies for "Fiber to the Home".

The FTTH generally adopted by operators is a PON (Passive Optical Network) architecture. The PON is a point-to-multipoint architecture based on the following elements:
- Shared fiber optic infrastructure. The use of optical couplers in the network is the basis of the architecture and deployment engineering. The couplers are used to serve several zones or several subscribers.
- Central equipment acting as Optical Line Termination (OLT). The OLT manages the broadcasting and reception of streams through network interfaces. It receives signals from subscribers and broadcasts a content based on specific services. 
- End equipments:
    - ONT (Optical Network Terminations) in case where the equipment is dedicated to a customer and the fiber reaches the customer. This is then an FTTH (Fiber To The Home) type architecture. There is only one fiber per customer (signals are bidirectional).
    - ONU (Optical Network Unit) in the case where the equipment is dedicated to a whole building. This is then an FTTB (Fiber To The Building).
    
<img src="https://image.makewebeasy.net/makeweb/0/p4Ky6EVg4/optical%20fiber-knowledge/Apps_FTTx_Fig3.png">

The data for this challenge is collected from sensors at the ONT level. 

### The data

The data is coming from two different cities: city A (the source) and city B (the target). Data is labeled for city A but (mostly) unlabeled for city B (only 20% of labeled data is known for city B). For both cities A and B, the data is a time series collected for abuout 60 days. The granularity of the time series is 15 minutes. The samples reprensent different users (thus different ONT). At each time step, we have a ten-dimensional measurement of the following features (in parenthesis are the units of each feature).
- features: 
  - current: bias current of the GPON (Gigabit Passive Optical Network) ONT optical module (mA)
  - err_down_bip: number of ONT downstream frames with BIP error (integer)
  - err_up_bip: number of ONT upstream frames with BIP error (integer)
  - olt_recv: receiving power of the GPON ONT optical module from the ONU (dBm)
  - rdown: downstream rate of GPON ONT (Mbs)
  - recv: receiving power of the GPON ONT optical module (dBm)
  - rup: upstream rate of GPON ONT (Mbs)
  - send: transmitting power of the GPON ONT optical module (dBm)
  - temp: temperature of the GPON ONT optical module (Celsius)
  - volt: power feed voltage of GPON ONT optical module (mV)
- labels: 0 (weak) or 1 (failure) for sample. 

The goal of the challenge is to separate weak from failure, the good data are just given as side information (could be used for calibration), thus the goal is to submit a binary classifier. 

Let $x_t$ be the sample collected at the day $t$, then the label corresponding is computed on the day $t+7$. We aim to predict a failure from data coming from 7 days before.  

The data is given to you with shape **[users, timestamps, features]** and the features are given in the same order as presented above. For each user and timestamp, we aggregate seven days of data.

Note that the public data set (given to you with the starting kit) and the private data set (used to evaluate your submissions on the server) come from the same distribution, so in principle you could use the labeled public target data to learn a classifier and submit the actual function. This would defeat the purpose of transfer learning, so we decided to slightly but significantly transform the private data set to make this strategy non-performing.

#### Missing data

You will notice that a some data is missing in the datasets. There may be several reasons:

1. No data was gathered on a specific date for a specific user.
2. The data collecting process fails to retrieve a feature.
    
It is part of the challenge to overcome this real-life difficulty.

### The scoring metrics

In this challenge we evaluate the performance using five different metrics:

- Accuracy (**acc**): The number of correctly predicted labels over the total number of the samples [sklearn function](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score). 
- Area unther the ROC curve (**auc**). This score gives us the probability that a failure instance is scored higher than a weak instance by the binary discriminant function [sklearn function](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html).
- Average precision (**ap**): it summarizes a precision-recall curve as the weighted mean of precisions achieved at each threshold, with the increase in recall from the previous threshold used as the weight: $\sum_n (R_n - R_{n-1}) P_n$ where $P_n$ and $R_n$ are the precision and the recall at the $n$-th treshold [sklearn function](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html#sklearn.metrics.average_precision_score).
- **Precision@Recall**: It is a hybrid score implemented in `utils.scores`. It computes the precision when the recall is at some percentage, that is, rec@k is the the precision when the recall is at k%.

**Note that the average precision (ap) is the official metric used for the final evaluation**.


## Competition rules

- The competition will start on June 16, 2021 at 18h UTC (20h in Paris) and end on June 27, 2021 at 18h UTC (20h in Paris).
- Members are eligible to this competition if they are registered currently or were registered and graduated not earlier than 2019 in a higher education institute in France, Spain, Portugal, or Luxembourg. 
- Eligible Members may sign up to the competition, to become a Participant. A Participant will be initially assigned to his/her special individual team. If Participants wish, they may form Teams of maximum four members using the “my teams” view in their account, before making any submission. Participants can leave and join Teams freely, under the limit of four members, as long as they, or the Team they would like to join, has not yet made a submission. Once a Team (individual or multiple member) makes a Submission, it becomes immutable. Any Participant can leave a Team, which has not yet made a Submission, on his/her own accord. Joining a Team will require the approval of all existing Team Members.
- All models will be trained on the same cloud server allowing 16 CPUs (with shared memory of 128GB RAM).
- Teams will be given a total of 20 machine hours per cross validation fold. Submissions of a given Team will be ordered by Submission timestamp. We will make an attempt to train all Submissions, but starting from (and including) the first Submission that makes the Team’s total training time exceed 20 hours, all Submissions will be disqualified from the competition. Testing time will not count towards the limit. Training time will be displayed on the leaderboard for all Submissions, rounded to second. If a Submission raises an exception, its training time will not count towards the total.
- There is a timeout of 24 hours between Submissions. The only exception is when a Submission cannot be trained due to an error. In this case the Team can resubmit a Submission under the same name immediately.
- Submissions submitted after the end of the competition will not qualify for prizes.
- The public leaderboard will display validation scores running a cross-validation. The official scores will be calculated on the hidden test set and will be published after the closing of the competition. We will rank Submissions according to their Average Precision score.
- The organizers will do their best so that the provided backend runs flawlessly. We will communicate with participants in case of concerns and will try to resolve all issues, but we reserve the right to make unilateral decisions in specific cases, not covered by this set of minimal rules.
- The organizers reserve the right to disqualify any participant found to violate the fair competitive spirit of the Data Challenge. Possible reasons, without being exhaustive, are multiple accounts, attempts to access the test data, etc.
- Participants retain copyright on their submitted code and grant reuse under BSD 3-Clause License.

Participants accept these rules automatically when signing up to the “Transfer learning for detecting failures of optical access network” Data Challenge.



## Getting started


Besides the usual pydata libraries, you will need to install `ramp-workflow`:
```
pip install git+https://github.com/paris-saclay-cds/ramp-workflow.git
```

It will install the `rampwf` library and the `ramp-test` script that you can use to check your submission before submitting. You do not need to know this package for participating in the challenge, but it could be useful to take a look at the [documentation](https://paris-saclay-cds.github.io/ramp-docs/ramp-workflow/advanced/index.html) if you would like to know what happens when we test your model, especially the [RAMP execution](https://paris-saclay-cds.github.io/ramp-docs/ramp-workflow/advanced/scoring.html) page to understand `ramp-test`, and the [commands](https://paris-saclay-cds.github.io/ramp-docs/ramp-workflow/advanced/command_line.html) to understand the different command line options. 

In [1]:
import numpy as np
import rampwf as rw

Read `problem.py` so you can have an access to the same interface as the testing script.

In [2]:
problem = rw.utils.assert_read_problem()

### The data

First take the public data set from the #oan_failure_challenge channel of the Slack team (join by [clicking here](https://join.slack.com/t/huaweiramp/shared_invite/zt-qbf4vy9s-0NS4~V898h40x8cI2KHEfQ)) and unzip it to create `./data`, then execute the `prepare_data.py` script in `./data`. Note that the public data given to you is different from the private data used to evaluate your submissions at the server.

The training data is composed of source and target data coming respectively from city A and city B. In real life, the FTTH problem has three classes: 1) the flow is normal and everything is going smoothly (good), 2) the flow is poor but the conection still working (weak), and 3) failure. For the OAN failure detection we are interested in a binary classification between the two classes: \[weak, failure\]. You are free to exploit the data of the good class but in the scoring you are only judged on the binary classification. 

The dataset you are given is composed of:
- The source data:
    - `X_train.source`: Data for the weak and failure classes.
    - `X_train.source_bkg`: Data for the good class.
    - `y_train.source`: Labels for `X_train.source`, 0: weak and 1: failure.
- The target data: 
    - `X_train.target`: (Labeled) target data for the weak and failure classes.
    - `X_train.target_unlabeled`: Unlabeled target data.
    - `X_train.target_bkg`: Target data for the good class.
    - `y_train.target`: Labels for `X_train.target`, 0: weak and 1: failure.
    
Since we are interested in the performance of the classifier on the target data, the test set is composed entirely of target data. `predict` will receive both `X_test.target` and `X_test.target_bkg`, and expected to produce probabilities of the weak and failure labels **only for the `X_test.target`**.

Read the training and test data.

In [3]:
X_train, y_train = problem.get_train_data()
X_test, y_test = problem.get_test_data()

Train data
Optical Dataset composed of
46110 source samples
50862 source background samples
438 target labeled samples
8202 target unlabeled samples
29592 target background samples
 Optical Dataset labels composed of
46110 labels of source samples
438 labels of target samples

Test data
Optical Dataset composed of
0 source samples
0 source background samples
17758 target labeled samples
0 target unlabeled samples
47275 target background samples
 Optical Dataset labels composed of
0 labels of source samples
17758 labels of target samples



The input data is three-dimensional (sample, time, features). Time has 672 dimensions (4 times an hour $\times$ 24 hours $\times$ 7 days). It contains nan values thus it should be cleaned.

In [4]:
X_train.source[7]

array([[1.30e+01, 0.00e+00, 0.00e+00, ..., 2.28e+00, 4.40e+01, 3.26e+03],
       [1.30e+01, 0.00e+00, 0.00e+00, ..., 2.23e+00, 4.40e+01, 3.26e+03],
       [1.30e+01, 0.00e+00, 0.00e+00, ..., 2.31e+00, 4.40e+01, 3.26e+03],
       ...,
       [     nan,      nan,      nan, ...,      nan,      nan,      nan],
       [     nan,      nan,      nan, ...,      nan,      nan,      nan],
       [     nan,      nan,      nan, ...,      nan,      nan,      nan]],
      dtype=float32)

In [5]:
X_train.source[10000]

array([[1.4000e+01, 5.3560e+03, 0.0000e+00, ..., 2.1500e+00, 4.7000e+01,
        3.3000e+03],
       [1.4000e+01, 6.2650e+03, 2.0000e+00, ..., 2.0800e+00, 4.7000e+01,
        3.3200e+03],
       [1.4000e+01, 7.8850e+03, 4.0000e+00, ..., 2.4600e+00, 4.7000e+01,
        3.3000e+03],
       ...,
       [1.4000e+01, 5.1556e+04, 6.0000e+00, ..., 2.3000e+00, 4.6000e+01,
        3.3200e+03],
       [1.4000e+01, 4.3742e+04, 2.0000e+01, ..., 1.9500e+00, 4.6000e+01,
        3.3000e+03],
       [1.4000e+01, 4.4794e+04, 2.6000e+01, ..., 2.2800e+00, 4.6000e+01,
        3.3000e+03]], dtype=float32)

### The classification task

You should submit a feature extractor and a classifier. The `transform` function of the feature extractor is executed on  every input data (target, source, bkg) and the resulting arrays are passed to both to the `fit` and the `predict` functions of the classifier. The feature extractor of the starting kit replaces nans by zero, and flattens the matrix to (sample, 6720).

In [6]:
# %load submissions/starting_kit/feature_extractor.py
import numpy as np

class FeatureExtractor:

    def __init__(self):
        pass

    def transform(self, X):
        # Deal with NaNs inplace
        np.nan_to_num(X, copy=False)
        # We flatten the input, originally 3D (sample, time, dim) to
        # 2D (sample, time * dim)
        X = X.reshape(X.shape[0], -1)
        return X


The starting kit implements a naive domain adaptation where the model (random forest) trained on the source is used to classify the target.

In [7]:
# %load submissions/starting_kit/classifier.py
from sklearn.ensemble import RandomForestClassifier
from utils.dataset import OpticalDataset, OpticalLabels

import numpy as np

class Classifier:

    def __init__(self):
        self.clf = RandomForestClassifier(
            n_estimators=2, max_depth=2, random_state=44, n_jobs=-1)

    def fit(self, X_source, X_source_bkg, X_target, X_target_unlabeled,
            X_target_bkg, y_source, y_target):
        self.clf.fit(X_source, y_source)

    def predict_proba(self, X_target, X_target_bkg):
        y_proba = self.clf.predict_proba(X_target)
        return y_proba


You can look at the workflow code at `external_imports/utils/workflow.py` to see exactly how your submissions are loaded and used. You can execute the training and prediction of your submission here in the notebook. When you run `ramp-test`, we do cross validation; here you use the full training data to train and test data to test. [This page](https://paris-saclay-cds.github.io/ramp-docs/ramp-workflow/advanced/scoring.html) gives you a brief overview of what happens behind the scenes when you run the `ramp-test` script.

In [8]:
trained_workflow = problem.workflow.train_submission('submissions/starting_kit', X_train, y_train)
y_test_pred = problem.workflow.test_submission(trained_workflow, X_test)

### The scores

We compute six scores on the classification. All scores are implemented in `external_imports.utils.scores.py` so you can look at the precise definitions there.
**The official score of the competition is ap.**

In [9]:
ap    = problem.score_types[0]
rec5  = problem.score_types[1]
rec10 = problem.score_types[2]
rec20 = problem.score_types[3]
acc   = problem.score_types[4]
auc   = problem.score_types[5]

In [10]:
print('ap test score    = {}'.format(ap(y_test.target, y_test_pred[:,1])))
print('rec5 test score  = {}'.format(rec5(y_test.target, y_test_pred[:,1])))
print('rec10 test score = {}'.format(rec10(y_test.target, y_test_pred[:,1])))
print('rec20 test score = {}'.format(rec20(y_test.target, y_test_pred[:,1])))
print('acc test score   = {}'.format(acc(y_test.target, y_test_pred.argmax(axis=1))))
print('auc test score   = {}'.format(auc(y_test.target, y_test_pred[:,1])))

ap test score    = 0.1626234044092243
rec5 test score  = 0.07541412115097046
rec10 test score = 0.1970357447862625
rec20 test score = 0.34132519364356995
acc test score   = 0.821939407590945
auc test score   = 0.586923967966097


### The cross validation scheme

We use a ten-fold shuffle split cross-validation (stratified when labels are available) for all data sets. In each fold, 20% of the instances are in the validation set, except for the labeled target data which serves mostly for validation (to obtain an unbiased estimate of the test scores, evaluated entirely on labeled target samples). We do put twenty labeled target points in the training folds. The rationale is that when we extend our broadband services to City B, we may obtain a small set of labeled data rapidly, but we would like to deploy our failure detector without waiting two months for collecting a data comparable to that of City A.

The cross-validation scheme (see `problem.get_cv`) is implemented in the `TLShuffleSplit` class of `external_imports.utils.cv.py`, if you want to take a closer look.

You are free to play with both the train/test cut and the cross-validation when developing your models but be aware that we will use the same setup on the official server as the one in the RAMP kit (on a different set of four campaigns that will not be available to you).

The following cell goes through the same steps as the official evaluation script (`ramp-test`).

In [11]:
splits = problem.get_cv(X_train, y_train)

y_test_preds = []
for fold_i, (train_is, valid_is) in enumerate(splits):
    trained_workflow = problem.workflow.train_submission(
        'submissions/starting_kit', X_train, y_train, train_is)
    X_fold_train = X_train.slice(train_is)
    X_fold_valid = X_train.slice(valid_is)
    
    y_train_pred = problem.workflow.test_submission(trained_workflow, X_fold_train)
    y_valid_pred = problem.workflow.test_submission(trained_workflow, X_fold_valid)
    y_test_pred = problem.workflow.test_submission(trained_workflow, X_test)
    print('-------------------------------------')
    print('training ap on fold {} = {}'.format(
        fold_i, ap(y_train.slice(train_is).target, y_train_pred[:,1])))
    print('validation ap on fold {} = {}'.format(
        fold_i, ap(y_train.slice(valid_is).target, y_valid_pred[:,1])))
    print('test ap on fold {} = {}'.format(fold_i, ap(y_test.target, y_test_pred[:,1])))
    
    y_test_preds.append(y_test_pred)

-------------------------------------
training ap on fold 0 = 0.30833333333333335
validation ap on fold 0 = 0.2637875964895809
test ap on fold 0 = 0.16218430339780684
-------------------------------------
training ap on fold 1 = 0.21250000000000002
validation ap on fold 1 = 0.2555942077788053
test ap on fold 1 = 0.16361016472786805
-------------------------------------
training ap on fold 2 = 0.2
validation ap on fold 2 = 0.29440601825201235
test ap on fold 2 = 0.1745388926023523
-------------------------------------
training ap on fold 3 = 0.7375
validation ap on fold 3 = 0.28218715512682335
test ap on fold 3 = 0.16904411795376056
-------------------------------------
training ap on fold 4 = 0.21250000000000002
validation ap on fold 4 = 0.24879604051634688
test ap on fold 4 = 0.16172210972525408
-------------------------------------
training ap on fold 5 = 0.4
validation ap on fold 5 = 0.30569665610952207
test ap on fold 5 = 0.16752441466614315
-------------------------------------
tr

We compute both the mean test score and the score of bagging your ten models. The official ranking will be determined by the bagged test score (on different data sets from the ones you have). Your public score will be the bagged validation score (the averaging is [slightly more complicated](https://github.com/paris-saclay-cds/ramp-workflow/blob/master/rampwf/utils/combine.py#L56) since we need to take care of the cross validation masks properly). 

In [12]:
bagged_y_pred = np.array(y_test_preds).mean(axis=0)
print('Mean ap score = {}'.format(
    np.mean([ap(y_test.target, y_test_pred[:,1]) for y_test_pred in y_test_preds])))
print('Bagged ap score = {}'.format(
    ap(y_test.target, np.array([y_test_pred for y_test_pred in y_test_preds]).mean(axis=0)[:,1])))

Mean ap score = 0.1670974152623759
Bagged ap score = 0.16968309345277285


## Example submissions

Besides the starting kit we give you two other example submissions. The feature extractor is the same in all three. `source_rf` is similar to the starting kit, just uses more and deeper trees, to obtain a better score. `target_rf` is another extreme submission that only uses the (few) labeled target training instance to learn a classifier. It has a slightly worse perfomance than `source_rf` which means that the source data does enhance the classifer even though the source and target distributions differ.

### Results:
|          | ap             | rec-5         | rec-10         | rec-20         | acc            |  auc           | 
|:---------|:--------------:|:-------------:|:--------------:|:--------------:|:--------------:|:--------------:|   
|source_rf | 0.191 ± 0.0026 | 0.073 ± 0.002 | 0.176 ± 0.0032 | 0.357 ± 0.0075 | 0.84 ± 0.0014  | 0.637 ± 0.0063 | 
|target_rf | 0.163 ± 0.0218 | 0.067 ± 0.0182| 0.138 ± 0.0339 | 0.272 ± 0.0537 | 0.813 ± 0.036  | 0.591 ± 0.0399 | 

The big transfer learning question to solve is: **how to combine the low-bias high-variance target data with the high-bias low-variance source data**. Other questions we're expecting to see answers to:
1. Can we do a better preprocessing (missing data amputation, using the time in a more intelligent way) in the feature extractor?
2. Normally the background data (good instances) does not participate in the scoring, but it can inform the classifier of the distribution shift. How to use this information the best?

## Local testing (before submission)

You submission will contain a `feature_extractor.py` implementing a FeatureExtractor class with a `transform` function (no `fit`) and a `classifier.py` implementing a Classifier class with a `fit` and `predict_proba` functions as in the starting kit. You should place it in the `submission/<submission_name>` folder in your RAMP kit folder. To test your submission, go to your RAMP kit folder in the terminal and type
```
ramp-test --submission <submission_name>
```
It will train and test your submission much like we did it above in this notebook, and print the foldwise and summary scores. You can try it also in this notebook:

In [ ]:
!ramp-test --submission target_rf

If you want to have a local leaderboard, use the `--save-output` option when running `ramp-test`, then try `ramp-show leaderboard` with different options. For example:
```
ramp-show leaderboard --mean --metric "['ap','auc']" --step "['valid','test']" --precision 3
```
and
```
ramp-show leaderboard --bagged --metric "['auc']"
```

RAMP also has an experimental hyperopt feature, with random grid search implemented. If you want to use it, type
```
ramp-hyperopt --help
```
and check out the example submission [here](https://github.com/ramp-kits/titanic/tree/hyperopt/submissions/starting_kit_h).

## Submission

1. First you will need to sign up at the [Huawei RAMP site](https://xianti.fr/sign_up). Your will be approved shortly by a system admin once your student status is verified.
2. You will then need a second sign-up, this time for the [OAN failure challenge](https://xianti.fr/events/oan_failure_2021_france). If your site sign-up was approved in the previous point, you should see a "Join event" button on the right of the top menu. This request will also be approved by a site admin.
3. Once you are signed up, you can form or join a team (be careful: you can only change teams while neither you nor the team you would like to join have submitted a solution) and start submitting (once a day). If you are happy with your local scores, copy-paste your submission at the [sandbox](https://xianti.fr/events/oan_failure_2021_france/sandbox), press "submit now", name your submission, then give credits to which other submission you used (in the competitive phase you will see only your own submissions in the list.
4. Your submission will be sent to train. It will either come back with an error or will be scored. You can follow the status at [my submissions](https://xianti.fr/events/oan_failure_2021_france/my_submissions).
5. If there is an error, click on the error to see the trace. You can resubmit a failed submission **under the same name**, this will not count in your daily quota.
6. There is no way to delete trained submissions. In exceptional cases we can stop a submission that hasn't been scored yet so you can resubmit. We strongly suggest to finish training at least one fold locally (using `ramp-test`) before submitting so you can estimate the training time.
7. You can follow the scores of the other participants at the [public leaderboard](https://xianti.fr/events/oan_failure_2021_france/leaderboard).
8. The public [competition leaderboard](https://xianti.fr/events/oan_failure_2021_france/competition_leaderboard) displays the top submission (according to the public score) of each participant. You can change which of your submission enters the competition by pulling out the top submission. Click on the particular submission at [my submissions](https://xianti.fr/events/oan_failure_2021_france/my_submissions) and click on the yellow button. The operation is reversible as many times you want, even after the competition deadline.

## Contact

You can contact the organizers in the Slack of the challenge, join by [clicking here](https://join.slack.com/t/huaweiramp/shared_invite/zt-qbf4vy9s-0NS4~V898h40x8cI2KHEfQ). 